In [12]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
import json

In [13]:
df = pd.read_csv("/content/yelp_labelled.txt", names=['sentence', 'label'], sep='\t')

In [14]:
nltk.download('stopwords')

# Mengubah seluruh text kedalam bentuk lowercase
df['sentence'] = df['sentence'].str.lower()
 
# Menghilangkan stopwords
stop_word = set(stopwords.words('english'))
 
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_word)]))
 
# Melakukan split dataset
sentence = df['sentence'].values
label = df['label'].values
 
sentence_train, sentence_test, label_train, label_test = train_test_split(sentence, label, test_size=0.2, shuffle=False)
 
# Membuat tokenisasi
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' # Filter untuk menghilangkan symbols
 
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>", filters=filt)
 
tokenizer.fit_on_texts(sentence_train)
 
# Menyimpan word_index kedalam sebuah file json
word_index = tokenizer.word_index
 
with open('word_index.json', 'w') as fp:
    json.dump(word_index, fp)
 
# Membuat sequences dan melakukan padding
train_sekuens = tokenizer.texts_to_sequences(sentence_train)
test_sekuens = tokenizer.texts_to_sequences(sentence_test)
 
train_padded = pad_sequences(train_sekuens,
                             maxlen=20,
                             padding='post',
                             truncating='post')
test_padded = pad_sequences(test_sekuens,
                            maxlen=20,
                            padding='post',
                            truncating='post')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Membuat model
model = tf.keras.Sequential([
    Embedding(2000, 20, input_length=20),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
 
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
 
# Train model
num_epochs = 30
history = model.fit(train_padded, label_train,
                    epochs=num_epochs,
                    validation_data=(test_padded, label_test),
                    verbose=1)

Epoch 1/30
25/25 [==============================] - 1s 10ms/step - loss: 0.6917 - accuracy: 0.5500 - val_loss: 0.7076 - val_accuracy: 0.2400
Epoch 2/30
25/25 [==============================] - 0s 4ms/step - loss: 0.6870 - accuracy: 0.5650 - val_loss: 0.7314 - val_accuracy: 0.2400
Epoch 3/30
25/25 [==============================] - 0s 4ms/step - loss: 0.6803 - accuracy: 0.5650 - val_loss: 0.7578 - val_accuracy: 0.2400
Epoch 4/30
25/25 [==============================] - 0s 4ms/step - loss: 0.6678 - accuracy: 0.5700 - val_loss: 0.7597 - val_accuracy: 0.2400
Epoch 5/30
25/25 [==============================] - 0s 5ms/step - loss: 0.6282 - accuracy: 0.6850 - val_loss: 0.6649 - val_accuracy: 0.6250
Epoch 6/30
25/25 [==============================] - 0s 4ms/step - loss: 0.5196 - accuracy: 0.8425 - val_loss: 0.6100 - val_accuracy: 0.7050
Epoch 7/30
25/25 [==============================] - 0s 4ms/step - loss: 0.3648 - accuracy: 0.9200 - val_loss: 0.5725 - val_accuracy: 0.7150
Epoch 8/30
25/25 [=

In [16]:
model.save("model.h5")

In [17]:
# Install tensorflowjs
!pip install tensorflowjs
 
# Convert model.h5 to model
!tensorflowjs_converter --input_format=keras model.h5 tfjs_model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files

In [20]:
files.download('/content/tfjs_model/group1-shard1of1.bin')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>